利用groupby解决分组分组排序和选择若干个最大值的问题

In [1]:
import pandas as pd

In [30]:
df = pd.DataFrame({"A": [2, 3, 5, 4], "B": ['a', 'b', 'b', 'a'], "C": [200801, 200902, 200704, 201003]})
df

,A,B,C
0,2,a,200801
1,3,b,200902
2,5,b,200704
3,4,a,201003


#### 按照某列分组后返回DataFrameGroupBy对象，不可视

In [3]:
df.groupby('B')

In [9]:
# 多列分组
df.groupby(['B','A'])

#### 对DataFrameGroupBy进行[]取列，返回DataFrameGroupBy或SeriesGroupBy对象

In [4]:
df.groupby('B')['A']    # SeriesGroupBy对象

In [6]:
df.groupby('B')[['A','C']]    # DataFrameGroupBy对象

#### groupby的by索引可以为外部list或series

In [7]:
# 类似于df.groupby('B')['A']
df['A'].groupby(df['B'])

#### groupy之后对组内元素求取统计值

In [8]:
df.groupby('B').mean()

,A,C
B,,
a,3,200902
b,4,200803


In [31]:
# 通过设置as_index=False， 保留原列中B
df.groupby('B', as_index=False).mean()

,B,A,C
0,a,3,200902
1,b,4,200803


In [10]:
# 各列中最大值
df.groupby('B').max()   

,A,C
B,,
a,4,201003
b,5,200902


In [20]:
# 针对不同的列选用不同的聚合方法
import numpy as np
df.groupby('B').agg({'A':np.mean, 'C':np.sum})

,A,C
B,,
a,3,401804
b,4,401606


In [21]:
# 对同一列选用不同的聚合方法
df.groupby('B')['A'].agg({'mena':np.mean, 'standard deviation':np.std})

/Users/guofei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,mena,standard deviation
B,,
a,3,1.414214
b,4,1.414214


In [22]:
# 对不同的列选用多种不同的聚合方法
df.groupby('B').agg({'A':[np.mean, 'sum'], 'C':['count', np.sum]})

A         C        
  mean sum count     sum
B                       
a    3   6     2  401804
b    4   8     2  401606

#### 分组后计数count和size

count计数时不包含NaN值，size计数时包含NaN值

In [32]:
df2 = pd.DataFrame({"Name":["Alice", "Bob", "Mallory", "Mallory", "Bob" , "Mallory"],
    "City":["Seattle", "Seattle", "Portland", "Seattle", "Seattle", "Portland"],
    "Val":[4,3,3,np.nan,np.nan,4]})
df2

,City,Name,Val
0,Seattle,Alice,4.0
1,Seattle,Bob,3.0
2,Portland,Mallory,3.0
3,Seattle,Mallory,NaN
4,Seattle,Bob,NaN
5,Portland,Mallory,4.0


In [35]:
# 不考虑Nan
df2.groupby(['Name', 'City'], as_index=False)['Val'].count()

,Name,City,Val
0,Alice,Seattle,1
1,Bob,Seattle,1
2,Mallory,Portland,2
3,Mallory,Seattle,0


In [42]:
# 考虑Nan
df2.groupby(['Name', 'City'])['Val'].size().reset_index(name='Size')

,Name,City,Size
0,Alice,Seattle,1
1,Bob,Seattle,2
2,Mallory,Portland,2
3,Mallory,Seattle,1


#### 分组运算apply

In [43]:
df

,A,B,C
0,2,a,200801
1,3,b,200902
2,5,b,200704
3,4,a,201003


In [47]:
df.groupby('B').apply(np.mean)

,A,C
B,,
a,3.0,200902.0
b,4.0,200803.0


In [52]:
df.groupby('B').apply(lambda x: x['C']-x['A'])

B   
a  0    200799
   3    200999
b  1    200899
   2    200699
dtype: int64

#### 组内取最大值

In [53]:
df3 = pd.DataFrame({"A": [2, 3, 5, 4], "B": ['a', 'b', 'b', 'a'], "C": [200801, 200902, 200704, 201003]})
df3

,A,B,C
0,2,a,200801
1,3,b,200902
2,5,b,200704
3,4,a,201003


In [54]:
# 返回值是一个带有multiindex的dataframe数据，其中level=0为groupby的by列，而level=1为原index
df3.groupby('B').apply(lambda x: x.sort_values('C', ascending=False))

A  B       C
B                
a 3  4  a  201003
  0  2  a  200801
b 1  3  b  200902
  2  5  b  200704

In [61]:
# 通过设置group_keys=False，不将group_keys作为index
df.groupby('B', group_keys=False).apply(lambda x: x.sort_values('C', ascending=False))

,A,B,C
3,4,a,201003
0,2,a,200801
1,3,b,200902
2,5,b,200704


In [58]:
# 通过reset_index去处multiindex
"""
level: 
drop: 是否放弃将原index放入columns中
"""
df3.groupby('B').apply(lambda x: x.sort_values('C', ascending=False)).reset_index(level=0,drop=True)

,A,B,C
3,4,a,201003
0,2,a,200801
1,3,b,200902
2,5,b,200704


In [60]:
# 通过再次group取其中的最大值
ddf3 = df3.groupby('B').apply(lambda x: x.sort_values('C', ascending=False)).reset_index(level=0,drop=True)
ddf3.groupby('B').first().reset_index()

,B,A,C
0,a,4,201003
1,b,3,200902


In [131]:
# 取每组中的前两个值
ddf3.groupby('B').head(2)

,A,B,C
3,4,a,201003
0,2,a,200801
1,3,b,200902
2,5,b,200704


In [138]:
# 另一种思路是先sort在groupby
ddf4 = df3.sort_values('C', ascending=False)
ddf4.groupby('B').head(1)

,A,B,C
3,4,a,201003
1,3,b,200902


#### 组内取top-K值

In [69]:
import random
df4 = pd.DataFrame(((random.randint(2012, 2016), random.choice(['tech', 'art', 'office']), '%dk-%dk'%(random.randint(2,10), random.randint(10, 20)), '') for _ in range(100)), columns=['publish_time', 'classf', 'salary', 'title'])
df4.head()

,publish_time,classf,salary,title
0,2015,art,8k-18k,
1,2013,art,3k-10k,
2,2014,tech,7k-13k,
3,2013,art,8k-10k,
4,2012,office,3k-13k,


In [103]:
df4.groupby(['publish_time', 'classf', 'salary']).count()['title'].groupby(level=0, group_keys=False).nlargest(5).reset_index()

,publish_time,classf,salary,title
0,2012,tech,8k-11k,2
1,2012,art,10k-13k,1
2,2012,art,2k-12k,1
3,2012,art,2k-15k,1
4,2012,art,2k-18k,1
5,2013,art,10k-14k,1
6,2013,art,3k-10k,1
7,2013,art,4k-11k,1
8,2013,art,4k-16k,1
9,2013,art,6k-13k,1


#### 按照bins进行分组

In [139]:
np.random.seed(0)
df5 = pd.DataFrame({'Age': np.random.randint(20, 70, 100), 
                   'Sex': np.random.choice(['Male', 'Female'], 100), 
                   'number_of_foo': np.random.randint(1, 20, 100)})
df5.head()

,Age,Sex,number_of_foo
0,64,Female,14
1,67,Female,14
2,20,Female,12
3,23,Male,17
4,23,Female,15


In [142]:
age_groups = pd.cut(df5['Age'], bins=[19,40,65,np.inf])
age_groups.head()

0    (40.0, 65.0]
1     (65.0, inf]
2    (19.0, 40.0]
3    (19.0, 40.0]
4    (19.0, 40.0]
Name: Age, dtype: category
Categories (3, interval[float64]): [(19.0, 40.0] < (40.0, 65.0] < (65.0, inf]]

In [143]:
df5.groupby(age_groups).mean()

,Age,number_of_foo
Age,,
"(19.0, 40.0]",29.840000,9.880000
"(40.0, 65.0]",52.833333,9.452381
"(65.0, inf]",67.375000,9.250000


In [146]:
pd.crosstab(df5['Sex'],age_groups)

Age,"(19.0, 40.0]","(40.0, 65.0]","(65.0, inf]"
Sex,,,
Female,22,18,3
Male,28,24,5
